# timeframe: 2011-02-21 to 2018-09-09

In [1]:
import pandas as pd
import numpy as np
import nltk
import spacy
from textblob import TextBlob

from nltk import word_tokenize, sent_tokenize
from nltk.corpus import words
from nltk.stem import SnowballStemmer
from nltk.stem import PorterStemmer

from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.decomposition import LatentDirichletAllocation

import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

import joblib
pd.options.display.colheader_justify = 'right'
pd.options.display.column_space = 1
pd.options.display.expand_frame_repr = True
pd.options.display.max_colwidth = 120

In [2]:
df = joblib.load('data/clean/clean_df.joblib')

In [ ]:
# don't use stemming with Word2Vec

In [3]:
df.shape

(45671, 10)

In [4]:
df.head(3)

,body,score,author,permalink,comment_id,created_loc,created_utc,post_title,post_score,post_id
0,thats an odd hashtag to throw at the end of this tweet,1085,RajasConCrema,/r/nba/comments/65wmkz/withers_lebron_said_he_couldnt_imagine_losing_a/dgdq1yr/,dgdq1yr,2017-04-18 00:37:33,2017-04-17 16:37:33,"[Withers] LeBron said he couldn't imagine losing a sibling like Isaiah Thomas did. ""I can only imagine how that woul...",1398,65wmkz
1,that situation is so tough already i cant imagine what it is like preparing all season to get ready for the playoffs...,84,NaN,/r/nba/comments/65wmkz/withers_lebron_said_he_couldnt_imagine_losing_a/dgdpz3b/,dgdpz3b,2017-04-18 00:36:07,2017-04-17 16:36:07,"[Withers] LeBron said he couldn't imagine losing a sibling like Isaiah Thomas did. ""I can only imagine how that woul...",1398,65wmkz
2,i dont think its brought up enough that he never had a sibling or a father figure the fact that hes a great team pla...,209,writingandshit,/r/nba/comments/65wmkz/withers_lebron_said_he_couldnt_imagine_losing_a/dgdsthu/,dgdsthu,2017-04-18 01:28:48,2017-04-17 17:28:48,"[Withers] LeBron said he couldn't imagine losing a sibling like Isaiah Thomas did. ""I can only imagine how that woul...",1398,65wmkz


In [148]:
# TODO: Properly do entire data set and a train_test_split later on
df_small = df[:20000]

df_small_train = df_small[:int(0.7 * df_small.shape[0])]
df_small_test = df_small[int(0.7 * df_small.shape[0]):]

In [48]:
test_cv= CountVectorizer(min_df = 10, max_df = .99, #ngram_range=(1, 2),
                         strip_accents='ascii',
                     strip_accents='unicode', encoding='utf-8')

In [49]:
dtm_tf = test_cv.fit_transform(df.body)
most_common_df = pd.DataFrame(dtm_tf.toarray(), columns=test_cv.get_feature_names())

In [50]:
df.body.str.contains('lebron').value_counts()

False    32504
True     13167
Name: body, dtype: int64

In [51]:
most_common_df.sum().sort_values(ascending=False)

the              41729
to               22576
and              19010
lebron           15172
is               14893
he               14514
in               14017
of               13472
that             12294
this             10117
it                9154
for               8322
his               7730
on                7698
was               7551
you               6337
be                6107
but               5796
with              5769
like              5581
just              4966
him               4852
not               4626
have              4571
at                4498
if                4440
as                4350
all               4347
game              4196
so                4134
                 ...  
memorable           10
opens               10
busy                10
butthurt            10
arsenal             10
obama               10
neutral             10
stellar             10
nailed              10
moon                10
stink               10
mismatch            10
minority   

In [12]:
from nltk.corpus import stopwords

# TF

In [92]:
def word_ngrams(self, tokens): #, stop_words=None):
    """Turn tokens into a sequence of n-grams after stop words filtering"""
    # handle stop words
#     if stop_words is not None:
#         tokens = [w for w in tokens if w not in stop_words]

    # handle token n-grams
    min_n, max_n = self.ngram_range
    if max_n != 1:
        original_tokens = tokens
        if min_n == 1:
            # no need to do any slicing for unigrams
            # just iterate through the original tokens
            tokens = list(original_tokens)
            min_n += 1
        else:
            tokens = []

        n_original_tokens = len(original_tokens)

        # bind method outside of loop to reduce overhead
        tokens_append = tokens.append
        space_join = " ".join

        for n in xrange(min_n,
                        min(max_n + 1, n_original_tokens + 1)):
            for i in xrange(n_original_tokens - n + 1):
                tokens_append(space_join(original_tokens[i: i + n]))

    return tokens

### Next steps:
1. I want to find the X topics/clusters about these texts. Perhaps it's fitness, politics, training, performance, GOAT.
2. Figure out which topic each document best corresponds to, and place the document within that topic/cluster
3. Chart a stacked-bar or a multi-line chart for each year (or month, or something else) from 2011-2018, and show
   the PERCENTAGE (Or COUNT) of each topic for that period of time.
4. Look at the insights! What does that mean?
5. Perhaps do sentiment analysis. See if I can find a well-trained model. Or train my own based on upvote? score? (Probably not doing this)

**NOTE**: Simply do LDA first, and make this work. Get a chart. Get some insights.
**THEN:** Make it repeatable. Get it in a function somehow. Try other Dim Red and Unclassifying and Topic modeling techniques. See what works best.

In [138]:
words_corpus = set(words.words())
analyzer = CountVectorizer().build_analyzer()
stem = SnowballStemmer('english')

stops = stopwords.words('english')
stops += ['just', 'lol', 'like', 'im', 'he', 'hes', 'would', 'get', 'going', 'doesnt', 'th', 'fuck', 'think', 'even', 'dont',
    'lebron', 'james', 'game', 'didnt', 'cant', 'say', 'see', 'look', 'go', 'said', 'also']
stops = set(stops)
acceptable_words = words_corpus - stops

Current isues:
* can't use stemmer and n_grams at same time.
* still too many junk words, even with growing stop_list
* better stop word list. perhaps from reddit.
* what is max_features?
* **don't have a lemmatizer in there. How can i do lemmatize, n_grams, stop words, and stemmer all in analyer?**

### Tell Roberto/Chad I'm having a real hard time cleaning my text

42862

In [175]:
wordpunct = nltk.WordPunctTokenizer()

In [177]:
lemmatizer = nltk.WordNetLemmatizer()

In [186]:
token_pos = nltk.pos_tag(wordpunct.tokenize('filthy'))

In [188]:
token_pos

[('filthy', 'NN')]

In [192]:
token_pos[0][1]

'NN'

In [195]:
lemmatizer.lemmatize('filthy', pos='adj')

KeyError: 'adj'

In [176]:
trythis = wordpunct.tokenize(df_small_train.body)

TypeError: expected string or bytes-like object

In [211]:
from nltk.tokenize import TreebankWordTokenizer


In [214]:
n_gram_size = 2

def english_corpus(doc, stemmer=stem):
    clean_words = [stemmer.stem(w) for w in analyzer(doc) if w in acceptable_words]
#     num_index = 0
#     for w in clean_words:
#         yield w
#         try:
#             if clean_words[num_index + 1]:
#                 yield clean_words[num_index + 1]
#         except:
#             continue
#         num_index+= 1    
    return [stemmer.stem(w) for w in analyzer(doc) if w in acceptable_words]

cv = CountVectorizer(stop_words=stops, 
                     #analyzer=english_corpus, 
                     min_df = 3, max_df = .95, ngram_range=(n_gram_size, n_gram_size),
                     strip_accents='unicode', 
                     encoding='utf-8',
                     tokenizer=TreebankWordTokenizer().tokenize,
                     #max_features=100
                    )
dtm_tf = cv.fit_transform(df_small_train.body)
dtm_tf.shape


# def words_and_char_bigrams(text):
# ...     words = re.findall(r'\w{3,}', text)
# ...     for w in words:
# ...         yield w
# ...         for i in range(len(w) - 2):
# ...             yield w[i:i+2]

/home/spencer/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:286: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo'] not in stop_words.
  sorted(inconsistent))


(14000, 3546)

In [215]:
# pd.DataFrame(dtm_tf.toarray(), columns=cv.get_feature_names()).shape

# lda_tf = LatentDirichletAllocation(n_components=10, random_state=0)
# lda_tf.fit(dtm_tf)

# # pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, cv)
# see_lda_topics(cv, 5, lda_tf, 7)

for n_topic in range(3,10):
    loop_lda_tf = LatentDirichletAllocation(n_components=n_topic, random_state=0)
    loop_lda_tf.fit(dtm_tf)
    print(':::::')
    print(f'Number of topics = {n_topic}')
    print(':::::')
    see_lda_topics(cv, n_topic, loop_lda_tf, 7)

:::::
Number of topics = 3
:::::
TOPIC 0
gon na
r nba
triple double
high school
got ta
locker room
free throw

TOPIC 1
best player
kevin love
holy shit
basketball player
anyone else
years ago
ty lue

TOPIC 2
last year
regular season
year old
ever seen
ive ever
k k
ive seen

:::::
Number of topics = 4
:::::
TOPIC 0
gon na
triple double
got ta
locker room
wan na
last night
pretty sure

TOPIC 1
best player
kevin love
next year
holy shit
anyone else
basketball player
ty lue

TOPIC 2
gon na
last year
regular season
ever seen
ive ever
year old
k k

TOPIC 3
r nba
high school
every time
free throw
ive seen
space jam
cavs fans

:::::
Number of topics = 5
:::::
TOPIC 0
gon na
triple double
ever seen
ive ever
paul george
pretty sure
jr smith

TOPIC 1
best player
next year
holy shit
anyone else
god damn
player league
really good

TOPIC 2
gon na
regular season
last year
year old
k k
averaged ppg
free throws

TOPIC 3
r nba
every time
free throw
ive seen
high school
space jam
everyone else

TOPIC 4
g

KeyboardInterrupt: 

# TF-IDF

In [71]:
cv2 = CountVectorizer(#stop_words=stops, 
                     analyzer=english_corpus, 
                     min_df = 2, max_df = .95, #ngram_range=(1, 2),
                     strip_accents='unicode', encoding='utf-8')

tfidf_vectorizer = TfidfVectorizer(**cv2.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(df_small_train.body)

lda_tfidf = LatentDirichletAllocation(n_components=10, random_state=0)
lda_tfidf.fit(dtm_tfidf)

pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)

/home/spencer/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1547: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. <class 'numpy.int64'> 'dtype' will be converted to np.float64.
  UserWarning)
/home/spencer/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.051493 -0.005656       1        1  17.891667
7     -0.068342  0.014101       2        1  13.069484
5     -0.035743 -0.013699       3        1   9.888963
4     -0.070377 -0.058975       4        1   9.431428
1     -0.082874 -0.050363       5        1   9.166373
6      0.089967  0.041093       6        1   8.382241
3     -0.002557  0.129411       7        1   8.372704
2      0.082768  0.092074       8        1   8.316884
9     -0.001495 -0.011377       9        1   8.013487
8      0.140146 -0.136609      10        1   7.466769, topic_info=     Category       Freq      Term      Total  loglift  logprob
term                                                           
1510  Default  34.000000      nice  34.000000  30.0000  30.0000
491   Default  33.000000   confirm  33.000000  29.0000  29.0000
593   Default  46.000000      damn  46.000000  28.0000  28.0000
696   Default  22.000000     doubl  22.000000  27.0000  27.0000
1261  Default  20.000000      king  20.000000  26.0000  26.0000
1458  Default  18.000000      miss  18.000000  25.0000  25.0000
1390  Default  65.000000       man  65.000000  24.0000  24.0000
2403  Default  16.000000     tripl  16.000000  23.0000  23.0000
1382  Default  24.000000     magic  24.000000  22.0000  22.0000
1897  Default  42.000000     right  42.000000  21.0000  21.0000
1552  Default  20.000000        oh  20.000000  20.0000  20.0000
1010  Default  26.000000       god  26.000000  19.0000  19.0000
2331  Default  14.000000       tho  14.000000  18.0000  18.0000
1367  Default  65.000000      love  65.000000  17.0000  17.0000
1446  Default  15.000000   million  15.000000  16.0000  16.0000
1985  Default  54.000000    season  54.000000  15.0000  15.0000
1781  Default  18.000000  question  18.000000  14.0000  14.0000
1842  Default  15.000000   regular  15.000000  13.0000  13.0000
1045  Default  43.000000       guy  43.000000  12.0000  12.0000
2382  Default  15.000000     trade  15.000000  11.0000  11.0000
445   Default  25.000000     coach  25.000000  10.0000  10.0000
1614  Default  27.000000      pass  27.000000   9.0000   9.0000
1830  Default  14.000000    record  14.000000   8.0000   8.0000
1875  Default  16.000000   respect  16.000000   7.0000   7.0000
336   Default  28.000000      call  28.000000   6.0000   6.0000
1415  Default  24.000000      mean  24.000000   5.0000   5.0000
103   Default  23.000000     anyon  23.000000   4.0000   4.0000
1278  Default  17.000000      lanc  17.000000   3.0000   3.0000
2358  Default  15.000000     today  15.000000   2.0000   2.0000
28    Default  11.000000        ad  11.000000   1.0000   1.0000
...       ...        ...       ...        ...      ...      ...
136   Topic10   1.836326  assassin   2.601165   2.2465  -6.4022
1401  Topic10   1.772000    martin   2.536830   2.2359  -6.4379
1462  Topic10   2.239007        mo   3.221734   2.2308  -6.2040
1515  Topic10   6.359971    nobodi   9.640802   2.1787  -5.1600
1875  Topic10  10.100956   respect  16.101310   2.1284  -4.6974
2550  Topic10   5.587835      west   8.584732   2.1653  -5.2894
736   Topic10   3.532103     earth   5.281132   2.1925  -5.7481
1252  Topic10   3.969750       kat   6.067462   2.1705  -5.6313
1278  Topic10   9.606736      lanc  17.530202   1.9932  -4.7475
103   Topic10  11.903119     anyon  23.768682   1.9031  -4.5332
2250  Topic10   6.523186     super  12.347907   1.9566  -5.1346
2522  Topic10   3.774530      wall   6.325863   2.0783  -5.6817
212   Topic10   5.995098    behind  12.764355   1.8390  -5.2191
1312  Topic10   4.452740     legit   8.572849   1.9396  -5.5165
2258  Topic10   4.457640   surpris   8.655915   1.9311  -5.5154
1266  Topic10  10.888165      know  47.357496   1.1247  -4.6223
740   Topic10   4.665654    easili   9.482974   1.8854  -5.4698
1110  Topic10   4.447925      hold   9.409466   1.8454  -5.5176
765   Topic10   5.365001       els  17.279312

In [120]:
def see_lda_topics(vectorizer, n_topics, fit_lda, n_examples):
    '''
    Prints top 10 names assigned to the topics, and the topics themselves
    '''
    vocab = vectorizer.get_feature_names()
    for topic in range(n_topics):
        print(f"TOPIC {topic}")
        for j in np.argsort(-fit_lda.components_,1)[topic,:n_examples]:
            print(vocab[j])
        print()
see_lda_topics(cv, 10, lda_tf, 3)

TOPIC 0
game
cavs
got

TOPIC 1
nice
damn
think

TOPIC 2
dont
see
game

TOPIC 3
game
nba
one

TOPIC 4
player
finals
team

TOPIC 5
team
think
player

TOPIC 6
one
fucking
people

TOPIC 7
double
really
seasons

TOPIC 8
man
good
confirmed

TOPIC 9
game
games
season



In [23]:
cv.get_feature_names()

['abil',
 'abl',
 'about',
 'abov',
 'absolut',
 'absorb',
 'absurd',
 'abus',
 'accept',
 'accomplish',
 'accord',
 'achiev',
 'across',
 'act',
 'activ',
 'actor',
 'actual',
 'ad',
 'add',
 'adjust',
 'admit',
 'advanc',
 'advantag',
 'advertis',
 'advic',
 'advoc',
 'after',
 'again',
 'against',
 'age',
 'agenc',
 'agent',
 'aggress',
 'ago',
 'agre',
 'ah',
 'ahead',
 'ai',
 'aint',
 'air',
 'albeit',
 'alien',
 'all',
 'alley',
 'almost',
 'alon',
 'along',
 'alongsid',
 'alreadi',
 'also',
 'altern',
 'alway',
 'am',
 'amaz',
 'ami',
 'among',
 'amongst',
 'amount',
 'an',
 'analysi',
 'analyst',
 'and',
 'angl',
 'angri',
 'ani',
 'anim',
 'ankl',
 'annoy',
 'annual',
 'anonym',
 'anoth',
 'answer',
 'anti',
 'anybodi',
 'anyon',
 'anyth',
 'anyway',
 'anywher',
 'apart',
 'appear',
 'appl',
 'appreci',
 'appropri',
 'are',
 'arena',
 'argu',
 'argument',
 'arm',
 'around',
 'art',
 'articl',
 'as',
 'asham',
 'asid',
 'ask',
 'aspect',
 'ass',
 'assassin',
 'assist',
 'assum'

In [25]:
cv = CountVectorizer(stop_words='english')
X = cv.fit_transform(sentences[:500])
Z = pd.DataFrame(X.toarray(),columns=cv.get_feature_names())

In [26]:
Z.shape

(500, 1790)

In [27]:
cv2 = CountVectorizer(stop_words='english')
XX = cv2.fit_transform(words[:500])
Z = pd.DataFrame(XX.toarray(),columns=cv2.get_feature_names())

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(stop_words='english')
X3 = tf.fit_transform(sentences[:500])
Z = pd.DataFrame(X3.toarray(), columns = tf.get_feature_names())

In [34]:
Z.shape

(500, 1790)

In [18]:
from nltk import sent_tokenize
text = df.body.apply(sent_tokenize)
text.sample(3, random_state=42)

25798    [As much hate as this dude gets, their is no denying that he can ball, and he's breaking these records at a relative...
26644                [The last 9 finals have consisted of either LeBron James Jones or Kobe Bryant, but never at the same time.]
5307                                                                                                        [That's just unfair]
Name: body, dtype: object

In [19]:
from nltk import word_tokenize
words = df.body.apply(word_tokenize)
words.sample(3, random_state=42)    

25798    [As, much, hate, as, this, dude, gets, ,, their, is, no, denying, that, he, can, ball, ,, and, he, 's, breaking, the...
26644    [The, last, 9, finals, have, consisted, of, either, LeBron, James, Jones, or, Kobe, Bryant, ,, but, never, at, the, ...
5307                                                                                                    [That, 's, just, unfair]
Name: body, dtype: object

In [32]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')
X = cv.fit_transform(df.body[90:100])
pd.DataFrame(X.toarray(),columns=cv.get_feature_names())

,16,added,ago,akron,april,believe,better,chugging,come,considered,...,retired,retires,rose,said,smhhhh,teenager,think,wait,workaholic,years
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,1,0,1,...,1,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
5,0,0,0,0,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,1,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
7,1,0,1,1,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
8,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0
9,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [2]:
df = joblib.load('data/clean/clean_df.joblib')

In [25]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

sia = SIA()

In [21]:
prob_selected = 2/16
num_trial = 27

In [26]:
# probability of num of trials up to and until 1st success
P = ((1-prob_selected)**(num_trial-1)) * prob_selected
P * 100

0.3882570868012486

In [25]:
# probability of num of failures until 1st success
P2 = ((1-prob_selected)**(num_trial)) * prob_selected
P2 * 100

0.33972495095109256